In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG

%reload_ext autoreload
%autoreload 2

In [2]:
sys.path.insert(0, os.path.join(CONFIG.CFG.BASEPATH, "Yet-Another-EfficientDet-Pytorch"))

In [3]:
%%capture
import subprocess
subprocess.call("git clone https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch", shell=True, cwd=CONFIG.CFG.BASEPATH)

0

In [3]:
import torch
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from backbone import EfficientDetBackbone
from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, postprocess, invert_affine
from torch.backends import cudnn

In [4]:
compound_coef = 4
force_input_size = None
img_path = [os.path.join("../../data", "test", img_path) for img_path in os.listdir(os.path.join(CONFIG.CFG.DATA.BASE, "test"))]
# IMG_PATH = os.path.join(CONFIG.CFG.DATA.BASE, "test", "2fd875eaa.jpg")

threshold = 0.2
iou_threshold = 0.2

use_cuda = True
use_float16 = False
cudnn.fastest = True
cudnn.benchmark = True

obj_list = ['wheat']

In [5]:
%%capture
# tf bilinear interpolation is different from any other's, just make do
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size
ori_imgs, framed_imgs, framed_metas = preprocess(*img_path, max_size=input_size)

if use_cuda:
    x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
else:
    x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),

                             # replace this part with your project's anchor config
                             ratios=[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)],
                             scales=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

model.load_state_dict(torch.load('efficientdet-d4_14_20235.pth'))
model.requires_grad_(False)
model.eval()

In [6]:
if use_cuda:
    model = model.cuda()
if use_float16:
    model = model.half()

In [30]:
with torch.no_grad():
    out = []
    for i in range(0, x.shape[0], 3):
        features, regression, classification, anchors = model(x[i:i+3])

        regressBoxes = BBoxTransform()
        clipBoxes = ClipBoxes()

        ou = postprocess(x[i:i+3],
                        anchors, regression, classification,
                        regressBoxes, clipBoxes,
                        threshold, iou_threshold)
        out += ou

out = invert_affine(framed_metas, out)

In [31]:
for i in range(len(ori_imgs)):
    print(img_path[i].split("/")[-1][:-4])
    if len(out[i]['rois']) == 0:
        continue
    
    for j in range(len(out[i]['rois'])):
        score = float(out[i]['scores'][j])
        (x1, y1, x2, y2) = out[i]['rois'][j].astype(np.int)
        print(score, x1, y1, x2-x1, y2-y1)
        cv2.rectangle(ori_imgs[i], (x1, y1), (x2, y2), (255, 255, 0), 2)
        obj = obj_list[out[i]['class_ids'][j]]

        cv2.putText(ori_imgs[i], '{}, {:.3f}'.format(obj, score),
                    (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 255, 0), 1)
    print()
        # plt.imshow(ori_imgs[i])

51f1be19e
0.9052724242210388 607 81 162 174
0.9031292796134949 269 463 147 124
0.886219322681427 838 262 136 198
0.8541731238365173 29 0 81 70
0.8376290798187256 772 880 147 102
0.7843184471130371 809 758 104 93
0.7794938683509827 67 687 136 222
0.7763395309448242 339 122 109 197
0.7422183752059937 498 461 207 109
0.7362051606178284 251 117 82 114
0.7208573222160339 696 915 83 84
0.7110490202903748 818 87 105 86
0.7095747590065002 651 767 112 102
0.6785475015640259 0 372 58 134
0.6432065367698669 189 918 108 103
0.6431254148483276 879 654 132 108
0.6083873510360718 566 590 115 124
0.5237424373626709 659 580 112 97
0.5048679113388062 515 978 108 45
0.34344282746315 671 437 69 51
0.25036996603012085 711 738 100 62
0.241500124335289 845 994 81 29
0.2290305346250534 864 417 90 50

cc3532ff6
0.9673125743865967 773 824 160 159
0.9494174122810364 74 799 136 171
0.9384627342224121 266 636 97 168
0.9293781518936157 95 604 87 146
0.9224253296852112 560 301 113 109
0.9198462963104248 377 1 81 96
